In [1]:
# default_exp runs

In [2]:
import sys

In [3]:
sys.path.append('/workspace/oct_ca_seg/oct/oct/')

In [4]:
from oct.startup import *
from model import CapsNet
import numpy as np
import mlflow
from fastai.vision import *
import mlflow.pytorch as MLPY
from fastai.utils.mem import gpu_mem_get_all

In [5]:
gpu_mem_get_all()

[GPUMemory(total=16160, free=16148, used=12)]

### Configuration Setup

In [6]:
name = 'FCN'

In [7]:
config_dict = loadConfigJSONToDict('configCAPS_APPresnet18.json')
config_dict['LEARNER']['lr']= 0.001
config_dict['LEARNER']['bs'] = 16
config_dict['LEARNER']['epochs'] = 30
config_dict['LEARNER']['runsave_dir'] = '/workspace/oct_ca_seg/runsaves/'
config_dict['MODEL'] = 'FASTAI FCN RESNET18 BACKBONE NO PRETRAIN'
config = DeepConfig(config_dict)

In [8]:
config_dict

{'LEARNER': {'loss_alpha': 0.04,
  'loss_beta': 0.8,
  'bs': 16,
  'epochs': 30,
  'img_size': 256,
  'lr': 0.001,
  'num_workers': 1,
  'runsave_dir': '/workspace/oct_ca_seg/runsaves/'},
 'MODEL': 'FASTAI FCN RESNET18 BACKBONE NO PRETRAIN',
 'DATSET': {'path': '/workspace/oct_ca_seg/COCOdata',
  'annotations_path': '/workspace/oct_ca_seg/COCOdata/train/images/medium_set_annotations.json'}}

In [10]:
metrics = [sens, spec, dice, my_Dice_Loss, acc]

In [11]:
def saveConfigRun(dictiontary, run_dir, name):
    with open(run_dir/name, 'w') as file:
        json.dump(dictiontary, file)

## Dataset

In [12]:
cocodata_path = Path('/workspace/oct_ca_seg/COCOdata/')
train_path = cocodata_path/'train/images'
valid_path = cocodata_path/'valid/images'
test_path = cocodata_path/'test/images'

### For complete dataset

In [13]:
fn_get_y = lambda image_name: Path(image_name).parent.parent/('labels/'+Path(image_name).name)
codes = np.loadtxt(cocodata_path/'codes.txt', dtype=str)
tfms = get_transforms()
src = (SegCustomItemList
       .from_folder(cocodata_path, recurse=True, extensions='.jpg')
       .filter_by_func(lambda fname: Path(fname).parent.name == 'images', )
       .split_by_folder('train', 'valid')
       .label_from_func(fn_get_y, classes=codes))
src.transform(tfms, tfm_y=True, size=config.LEARNER.img_size)
data = src.databunch(cocodata_path,
                     bs=config.LEARNER.bs,
                     val_bs=2*config.LEARNER.bs,
                     num_workers = config.LEARNER.num_workers)
stats = [torch.tensor([0.2190, 0.1984, 0.1928]), torch.tensor([0.0645, 0.0473, 0.0434])]
data.normalize(stats);
data.c_in, data.c_out = 3, 2

### For converting Validation set into a mini set to experiment on

In [16]:
fn_get_y = lambda image_name: Path(image_name).parent.parent/('labels/'+Path(image_name).name)
codes = np.loadtxt(cocodata_path/'codes.txt', dtype=str)
tfms = get_transforms()
src = (SegCustomItemList
       .from_folder(test_path, recurse=True, extensions='.jpg')
       .filter_by_func(lambda fname: Path(fname).parent.name == 'images', )
       .split_by_rand_pct(0.9)
       .label_from_func(fn_get_y, classes=codes))
src.transform(tfms, tfm_y=True, size =config.LEARNER.img_size)
data = src.databunch(test_path,
                     bs=config.LEARNER.bs,
                     val_bs=2*config.LEARNER.bs,
                     num_workers = config.LEARNER.num_workers)
stats = [torch.tensor([0.2190, 0.1984, 0.1928]), torch.tensor([0.0645, 0.0473, 0.0434])]
data.normalize(stats);
data.c_in, data.c_out = 3, 2

### Fastai FCN

In [10]:
import torchvision

In [13]:
torchvision.models.segmentation.deeplabv3_resnet50(pretrained=False, num_classes=2, progress=False)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/abalaji/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


ImportError: FloatProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [14]:
torchvision.models.segmentation.fcn_resnet50(pretrained=False, num_classes=2, progress=False)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/abalaji/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


ImportError: FloatProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [64]:
run_dir = config.LEARNER.runsave_dir+'/'+name
#os.mkdir(run_dir)
exp_name = 'fastai_fcn'
mlflow_CB = partial(MLFlowTracker,
                    exp_name=exp_name,
                    uri='file:/workspace/oct_ca_seg/runsaves/fastai_experiments/mlruns/',
                    params=config.config_dict,
                    log_model=True,
                    nb_path="/workspace/oct_ca_seg/oct/02_caps.ipynb")
learner = cnn_learner(data = data,
                        base_arch=models.resnet18,
                       pretrained=False,
                       metrics = metrics,
                       callback_fns=mlflow_CB)

In [55]:
learner.model(data.one_batch()[0].cuda())

tensor([[-0.5683,  0.5234],
        [ 5.2498, -0.9694],
        [-1.9473,  1.8972],
        [-0.8260, -0.6898],
        [-1.6650, -0.5685],
        [-0.3591,  0.7867],
        [-0.1963,  1.2976],
        [ 0.2490, -0.8992],
        [-2.4534,  1.3153],
        [ 1.1536,  3.7790],
        [ 0.7956,  0.1926],
        [-2.9585,  5.9474],
        [ 2.0949, -4.5373],
        [-1.8697, -0.5463],
        [ 1.9236, -0.8141],
        [-1.0284,  0.7806]], device='cuda:0', grad_fn=<AddmmBackward>)

In [34]:
with mlflow.start_run():
    learner.fit_one_cycle(1, slice(config.LEARNER.lr), pct_start=0.9)
    MLPY.save_model(learner.model, run_dir+'/model')
    save_all_results(learner, run_dir, exp_name)
    saveConfigRun(config.config_dict, run_dir=Path(run_dir), name = 'configUNET_APPresnet18_bs16_epochs15_lr0.001.json')

epoch,train_loss,valid_loss,sens,spec,dice,my_Dice_Loss,acc,time


ValueError: Expected input batch_size (16) to match target batch_size (1048576).